In [0]:
from pyspark.sql import functions as F

In [0]:
df = spark.read.option('header',True).option('inferSchema',True).format('csv').load('/Volumes/workspace/adhoc/customer')

df.limit(5).display()

In [0]:
'''
in serverless compute we can not create the rdd here.

'''

In [0]:
df.count()


for cols in df.columns:
    null_check = df.filter(F.col(cols).isNull()).count()
    print("Column name >> ",cols,null_check)


In [0]:
df.display()

In [0]:
df = df.withColumnRenamed('Subscription Date','subscription_date')

In [0]:
def getPartition(df,num):
    if num ==3:
        df = df.withColumn('snapshot_year',F.year(F.col('subscription_date')))\
            .withColumn("snapshot_month",F.month(F.col('subscription_date')))\
            .withColumn("snapshot_day",F.dayofmonth(F.col('subscription_date')))
    else:
        None

    return df




In [0]:
df = getPartition(df,3)

In [0]:
lst = [str(c) for  c in df.columns if ' ' in c]

for cols in lst:
    df = df.withColumnRenamed(cols,cols.replace(" ","_"))
    

In [0]:

df.write.mode('overwrite')\
    .option("overwriteSchema", "true")\
    .partitionBy(['snapshot_year','snapshot_month'])\
    .format('delta').save("s3://databrickspr/databricks_small_practice/")



In [0]:
%sql
show partitions workspace.adhoc.customers;